# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,necochea,-38.5473,-58.7368,283.54,92,8,3.50,AR,1734417476
1,1,guane,22.2025,-84.0875,294.65,93,2,2.04,CU,1734417477
2,2,port-aux-francais,-49.3500,70.2167,278.87,80,13,15.40,TF,1734417480
3,3,bredasdorp,-34.5322,20.0403,292.08,52,87,4.43,ZA,1734417481
4,4,olonkinbyen,70.9221,-8.7187,270.53,62,84,4.44,SJ,1734417482


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points (
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color= 'City'
)
# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
vacay1 = city_data_df[city_data_df['Humidity'] > 70]
vacay2 = vacay1[vacay1['Cloudiness'] < 50]
vacay3 = vacay2[vacay2['Wind Speed'] < 5 ]
# Drop any rows with null values
# YOUR CODE HERE
vacay_clean = vacay3.dropna()
# Display sample data
# YOUR CODE HERE
vacay_clean








,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,necochea,-38.5473,-58.7368,283.54,92,8,3.50,AR,1734417476
1,1,guane,22.2025,-84.0875,294.65,93,2,2.04,CU,1734417477
26,26,stanley,54.8680,-1.6985,280.65,86,20,3.09,GB,1734417514
33,33,ancud,-41.8697,-73.8203,284.55,96,23,1.46,CL,1734417522
40,40,villa gesell,-37.2639,-56.9730,287.74,87,0,2.37,AR,1734417530
...,...,...,...,...,...,...,...,...,...,...
545,545,borogontsy,62.6683,131.1683,232.20,100,40,1.18,RU,1734418179
548,548,haiku-pauwela,20.9219,-156.3051,293.80,82,14,2.68,US,1734418183
550,550,mo i rana,66.3128,14.1428,270.07,93,20,0.51,NO,1734418185
562,562,bull savanna,17.8868,-77.5902,296.14,85,1,3.95,JM,1734418200


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = vacay_clean[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''


# Display sample data
# YOUR CODE HERE
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,necochea,AR,-38.5473,-58.7368,92,
1,guane,CU,22.2025,-84.0875,93,
26,stanley,GB,54.8680,-1.6985,86,
33,ancud,CL,-41.8697,-73.8203,96,
40,villa gesell,AR,-37.2639,-56.9730,87,
...,...,...,...,...,...,...
545,borogontsy,RU,62.6683,131.1683,100,
548,haiku-pauwela,US,20.9219,-156.3051,82,
550,mo i rana,NO,66.3128,14.1428,93,
562,bull savanna,JM,17.8868,-77.5902,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = { "apiKey": geoapify_key,
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = "accommodation.hotel"


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address =name_address.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
necochea - nearest hotel: Cabañas Río Quequén
guane - nearest hotel: No hotel found
stanley - nearest hotel: Hotel 52
ancud - nearest hotel: Cabañas
villa gesell - nearest hotel: Hotel El Faro
flying fish cove - nearest hotel: Christmas Island Lodge
arkhangel'sk - nearest hotel: Серафима
chonchi - nearest hotel: Hotel Huildín
badger - nearest hotel: Clarion Hotel & Suites
tiksi - nearest hotel: Арктика
conception bay south - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
kapa'a - nearest hotel: Pono Kai Resort
coquimbo - nearest hotel: Hotel Iberia
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
fomboni - nearest hotel: Mohéli La Plage
san ignacio de velasco - nearest hotel: Hotel Caparu
keflavik - nearest hotel: Núpan Deluxe
avarua - nearest hotel: Paradise Inn
st. john's - nearest hotel: Murray Premises Hotel
ballina - nearest hotel: Ballina Manor Boutique Hotel
tahe - nearest hotel: No hotel found
weno - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,necochea,AR,-38.5473,-58.7368,92,Cabañas Río Quequén
1,guane,CU,22.2025,-84.0875,93,No hotel found
26,stanley,GB,54.8680,-1.6985,86,Hotel 52
33,ancud,CL,-41.8697,-73.8203,96,Cabañas
40,villa gesell,AR,-37.2639,-56.9730,87,Hotel El Faro
...,...,...,...,...,...,...
545,borogontsy,RU,62.6683,131.1683,100,No hotel found
548,haiku-pauwela,US,20.9219,-156.3051,82,Inn At Mama's Fish House
550,mo i rana,NO,66.3128,14.1428,93,Scandic Meyergården
562,bull savanna,JM,17.8868,-77.5902,85,Junction Guest House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points (
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    hover_cols = ['Hotel Name','City','Country'],
    color= 'City'
)

# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)